<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier 
from sklearn.utils import shuffle

Открыл и изучил информацию о данных. Все столбцы в корректном формате и без пропусков за исключением столбца Tenure.

In [18]:
data = pd.read_csv('/datasets/Churn.csv')
data.info()
display(data.head(10))

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 10000 entries, 0 to 9999

Data columns (total 14 columns):

 #   Column           Non-Null Count  Dtype  

---  ------           --------------  -----  

 0   RowNumber        10000 non-null  int64  

 1   CustomerId       10000 non-null  int64  

 2   Surname          10000 non-null  object 

 3   CreditScore      10000 non-null  int64  

 4   Geography        10000 non-null  object 

 5   Gender           10000 non-null  object 

 6   Age              10000 non-null  int64  

 7   Tenure           9091 non-null   float64

 8   Balance          10000 non-null  float64

 9   NumOfProducts    10000 non-null  int64  

 10  HasCrCard        10000 non-null  int64  

 11  IsActiveMember   10000 non-null  int64  

 12  EstimatedSalary  10000 non-null  float64

 13  Exited           10000 non-null  int64  

dtypes: float64(3), int64(8), object(3)

memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


Избавился от столбца с нумерацией - он не дает никакой полезной информации и может негативно повлиять на точность предсказаний.

In [19]:
data = data.drop('RowNumber', axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 10000 entries, 0 to 9999

Data columns (total 13 columns):

 #   Column           Non-Null Count  Dtype  

---  ------           --------------  -----  

 0   CustomerId       10000 non-null  int64  

 1   Surname          10000 non-null  object 

 2   CreditScore      10000 non-null  int64  

 3   Geography        10000 non-null  object 

 4   Gender           10000 non-null  object 

 5   Age              10000 non-null  int64  

 6   Tenure           9091 non-null   float64

 7   Balance          10000 non-null  float64

 8   NumOfProducts    10000 non-null  int64  

 9   HasCrCard        10000 non-null  int64  

 10  IsActiveMember   10000 non-null  int64  

 11  EstimatedSalary  10000 non-null  float64

 12  Exited           10000 non-null  int64  

dtypes: float64(3), int64(7), object(3)

memory usage: 1015.8+ KB


Проверил данные на дубликаты, также проверил нет ли общих ID: возможно клиент закрывал счет и снова открыл в банке, в данных он мог бы повторяться.

In [20]:
display(data.duplicated().sum())
display(data['CustomerId'].duplicated().sum())

0

0

Неясно что делать с пропусками в данных. Это могут быть новые клиенты, а может быть простой ошибкой данных. Количество лет пользования банком может быть существенным признаком для предсказаний. Придется избавиться от этих строк.

In [21]:
display(data[data['Tenure'].isna()])
display(data['Tenure'].unique())
data = data.dropna()
data.info()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

<class 'pandas.core.frame.DataFrame'>

Int64Index: 9091 entries, 0 to 9998

Data columns (total 13 columns):

 #   Column           Non-Null Count  Dtype  

---  ------           --------------  -----  

 0   CustomerId       9091 non-null   int64  

 1   Surname          9091 non-null   object 

 2   CreditScore      9091 non-null   int64  

 3   Geography        9091 non-null   object 

 4   Gender           9091 non-null   object 

 5   Age              9091 non-null   int64  

 6   Tenure           9091 non-null   float64

 7   Balance          9091 non-null   float64

 8   NumOfProducts    9091 non-null   int64  

 9   HasCrCard        9091 non-null   int64  

 10  IsActiveMember   9091 non-null   int64  

 11  EstimatedSalary  9091 non-null   float64

 12  Exited           9091 non-null   int64  

dtypes: float64(3), int64(7), object(3)

memory usage: 994.3+ KB


Судя по уникальным значениям столбца Tenure, лучше изменить в нем тип данных.

In [22]:
data['Tenure'] = data.loc[:, 'Tenure'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>

Int64Index: 9091 entries, 0 to 9998

Data columns (total 13 columns):

 #   Column           Non-Null Count  Dtype  

---  ------           --------------  -----  

 0   CustomerId       9091 non-null   int64  

 1   Surname          9091 non-null   object 

 2   CreditScore      9091 non-null   int64  

 3   Geography        9091 non-null   object 

 4   Gender           9091 non-null   object 

 5   Age              9091 non-null   int64  

 6   Tenure           9091 non-null   int64  

 7   Balance          9091 non-null   float64

 8   NumOfProducts    9091 non-null   int64  

 9   HasCrCard        9091 non-null   int64  

 10  IsActiveMember   9091 non-null   int64  

 11  EstimatedSalary  9091 non-null   float64

 12  Exited           9091 non-null   int64  

dtypes: float64(2), int64(8), object(3)

memory usage: 994.3+ KB


Так как на данных мы планируем обучать модель, стоит избавиться от не влияющих на целевой признак параметров, чтобы не путать модель.

In [23]:
data = data.drop(['Surname', 'CustomerId'], axis = 1)

## Исследование задачи

Разбил данные на три выборки: обучающую, валидационную и тестовую.

In [24]:
features = data.drop(['Exited'], axis = 1)
target = data['Exited']
features_train, features_test_valid, target_train, target_test_valid = train_test_split(features, target, test_size = 0.4, random_state = 12345, stratify = data['Exited'])
features_valid, features_test, target_valid, target_test =train_test_split(features_test_valid, target_test_valid, test_size = 0.5, random_state = 12345, stratify = target_test_valid)

Преобразовал категориальные признаки методом OHE.

In [25]:
features_train = pd.get_dummies(features_train, drop_first=True)
features_valid = pd.get_dummies(features_valid, drop_first=True)
features_test = pd.get_dummies(features_test, drop_first=True)
features_train.info()
features_train.head()

<class 'pandas.core.frame.DataFrame'>

Int64Index: 5454 entries, 9723 to 2671

Data columns (total 11 columns):

 #   Column             Non-Null Count  Dtype  

---  ------             --------------  -----  

 0   CreditScore        5454 non-null   int64  

 1   Age                5454 non-null   int64  

 2   Tenure             5454 non-null   int64  

 3   Balance            5454 non-null   float64

 4   NumOfProducts      5454 non-null   int64  

 5   HasCrCard          5454 non-null   int64  

 6   IsActiveMember     5454 non-null   int64  

 7   EstimatedSalary    5454 non-null   float64

 8   Geography_Germany  5454 non-null   uint8  

 9   Geography_Spain    5454 non-null   uint8  

 10  Gender_Male        5454 non-null   uint8  

dtypes: float64(2), int64(6), uint8(3)

memory usage: 399.5 KB


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9723,526,32,7,125540.05,1,0,0,86786.41,0,0,1
1224,500,47,8,128486.11,1,1,0,179227.12,0,1,0
8377,802,40,4,0.00,2,1,1,81908.09,0,1,1
8014,731,39,2,126816.18,1,1,1,74850.93,0,1,0
2491,612,26,4,0.00,2,1,1,179780.74,0,1,0


Изучил баланс классов. Найден явный дисбаланс.

In [26]:
data['Exited'].value_counts()

0    7237
1    1854
Name: Exited, dtype: int64

Стандартизировал данные для масштабирования

In [27]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train_scaled = scaler.transform(features_train)
features_valid_scaled = scaler.transform(features_valid)
features_test_scaled = scaler.transform(features_test)

Обучил модель логистической регрессии и дамми модель для сравнения. Регрессия показала невысокий результат, но по крайней мере выше, чем дамми модель.

In [28]:
%%time
best_model = None
best_f1 = 0
for iter in range(5, 100):
    model = LogisticRegression(max_iter = iter, random_state = 12345, solver = 'liblinear')
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    if f1_score(target_valid, predictions) > best_f1:
        best_f1 = f1_score(target_valid, predictions)
        best_model = model
display('best_f1', best_f1)
display('best_model', best_model)
dummy_model = DummyClassifier()
dummy_model.fit(features_train, target_train)
dummy_prediction = dummy_model.predict(features_valid)
display(f1_score(target_valid, dummy_prediction))

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warnings.warn("Liblinear failed to converge, increase "

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warnings.warn("Liblinear failed to converge, increase "

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warnings.warn("Liblinear failed to converge, increase "

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warnings.warn("Liblinear failed to converge, increase "

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warning

'best_f1'

0.07637231503579953

'best_model'

LogisticRegression(max_iter=19, random_state=12345, solver='liblinear')

0.0

CPU times: user 8.91 s, sys: 9.37 s, total: 18.3 s

Wall time: 18.3 s


Можно сделать вывод, что модель плохо обучается из за дисбаланса классов, также возможно это не самая эффективная модель в данной задаче.

## Борьба с дисбалансом

Для начала попробовал уравновесить классы. Таким образом построил три различных модели: дерево решений, случайный лес и логистическую регрессию.

Дерево показало близкий к требуемому результат F1 метрики: 0.587. К сожалению, этого недостаточно.

In [29]:
best_btree_model = None
best_btree_f1 = 0
for depth in range(1,11):
    for split in range(2, 5):
        for leaf in range(1, 7):
            btree_model = DecisionTreeClassifier(max_depth = depth, min_samples_split = split, min_samples_leaf = leaf, random_state = 12345, class_weight='balanced')
            btree_model.fit(features_train, target_train)
            btree_predictions = btree_model.predict(features_valid)
            if f1_score(target_valid, btree_predictions) > best_btree_f1:
                best_btree_f1 = f1_score(target_valid, btree_predictions)
                best_btree_model = btree_model
display('best_btree_f1', best_btree_f1)
display('best_btree_model', best_btree_model)

'best_btree_f1'

0.5873015873015872

'best_btree_model'

DecisionTreeClassifier(class_weight='balanced', max_depth=6, min_samples_leaf=4,
                       random_state=12345)

Случайный лес добился нужного результата: 0.665 показатель F1 метрики. (Код выполняется около 8 минут)

In [30]:
%%time
best_bforest_model = None
best_bforest_f1 = 0
for trees in range(10, 101, 10):
    for depth in range(1, 16):
        for split in range(2, 4):
            for leaf in range(1, 7):
                bforest_model = RandomForestClassifier(n_estimators = trees, max_depth = depth, min_samples_split = split, min_samples_leaf = leaf, random_state = 12345, class_weight='balanced')
                bforest_model.fit(features_train, target_train)
                bforest_predictions = bforest_model.predict(features_valid)
                if f1_score(target_valid, bforest_predictions) > best_bforest_f1:
                    best_bforest_f1 = f1_score(target_valid, bforest_predictions)
                    best_bforest_model = bforest_model
display('best_bforest_f1', best_bforest_f1)
display('best_bforest_model', best_bforest_model)

'best_bforest_f1'

0.6649484536082474

'best_bforest_model'

RandomForestClassifier(class_weight='balanced', max_depth=12,
                       min_samples_leaf=6, n_estimators=50, random_state=12345)

CPU times: user 6min 51s, sys: 3.57 s, total: 6min 55s

Wall time: 6min 55s


В качетсве альтернативного варианта борьбы с дисбалансом классов воспользовался методом upsampling. Посчитал downsampling неактуальным, данные урежутся слишком сильно.

In [32]:
features_zeroes = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeroes = target_train[target_train == 0]
target_ones = target_train[target_train == 1]
features_train_upsampled = pd.concat([features_zeroes]+[features_ones]*4)
target_train_upsampled = pd.concat([target_zeroes]+[target_ones]*4)
features_train_upsampled, target_train_upsampled = shuffle(features_train_upsampled, target_train_upsampled, random_state = 12345)

Дерево решений с помощью метода upsampling не изменило результат.

In [33]:
best_upstree_model = None
best_upstree_f1 = 0
for depth in range(1,11):
    for split in range(2, 5):
        for leaf in range(1, 7):
            upstree_model = DecisionTreeClassifier(max_depth = depth, min_samples_split = split, min_samples_leaf = leaf, random_state = 12345)
            upstree_model.fit(features_train_upsampled, target_train_upsampled)
            upstree_predictions = upstree_model.predict(features_valid)
            if f1_score(target_valid, upstree_predictions) > best_upstree_f1:
                best_upstree_f1 = f1_score(target_valid, upstree_predictions)
                best_upstree_model = upstree_model
display('best_upstree_f1', best_btree_f1)
display('best_upstree_model', best_upstree_model)

'best_upstree_f1'

0.5873015873015872

'best_upstree_model'

DecisionTreeClassifier(max_depth=6, min_samples_leaf=4, random_state=12345)

Случайный лес с помощью метода upsampling показал достойный результат, но хуже, чем при помощи простого уравновешнивания классов: 0.655 показатель F1 метрики. (Код выполняется около 11 минут)

In [34]:
%%time
best_upsforest_model = None
best_upsforest_f1 = 0
for trees in range(10, 101, 10):
    for depth in range(1, 16):
        for split in range(2, 4):
            for leaf in range(1, 7):
                upsforest_model = RandomForestClassifier(n_estimators = trees, max_depth = depth, min_samples_split = split, min_samples_leaf = leaf, random_state = 12345)
                upsforest_model.fit(features_train_upsampled, target_train_upsampled)
                upsforest_predictions = upsforest_model.predict(features_valid)
                if f1_score(target_valid, upsforest_predictions) > best_upsforest_f1:
                    best_upsforest_f1 = f1_score(target_valid, upsforest_predictions)
                    best_upsforest_model = upsforest_model
display('best_upsforest_f1', best_upsforest_f1)
display('best_upsforest_model', best_upsforest_model)

'best_upsforest_f1'

0.654891304347826

'best_upsforest_model'

RandomForestClassifier(max_depth=14, min_samples_split=3, n_estimators=30,
                       random_state=12345)

CPU times: user 9min 20s, sys: 3.04 s, total: 9min 23s

Wall time: 9min 23s


Логистическая регрессия также ухудшила результат F1 метрики: 0.463 (Код выполняется около 18 секунд)

In [35]:
%%time
best_upsregression_model = None
best_upsregression_f1 = 0
for iter in range(5, 100):
    upsregression_model = LogisticRegression(max_iter = iter, random_state = 12345, solver = 'liblinear')
    upsregression_model.fit(features_train_upsampled, target_train_upsampled)
    upsregression_predictions = upsregression_model.predict(features_valid)
    if f1_score(target_valid, upsregression_predictions) > best_upsregression_f1:
        best_upsregression_f1 = f1_score(target_valid, upsregression_predictions)
        best_upsregression_model = upsregression_model
display('best_upsregression_f1', best_upsregression_f1)
display('best_upsregression_model', best_upsregression_model)

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warnings.warn("Liblinear failed to converge, increase "

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warnings.warn("Liblinear failed to converge, increase "

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warnings.warn("Liblinear failed to converge, increase "

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warnings.warn("Liblinear failed to converge, increase "

/opt/conda/lib/python3.9/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.

  warning

'best_upsregression_f1'

0.46328671328671334

'best_upsregression_model'

LogisticRegression(max_iter=14, random_state=12345, solver='liblinear')

CPU times: user 9.24 s, sys: 9.51 s, total: 18.7 s

Wall time: 18.7 s


Исследовав модели, лучше всех себя показал случайный лес с уравновешенными классами. Чтобы еще улучшить результат, попробовал изменить порог классификации. Оказалось что лучший результат достигается при пороге 0.51. F1 метрика 0.668

По показателю AUC-ROC можно понять, что модель гораздо лучше случайной.

In [36]:
model = best_bforest_model
model.fit(features_train, target_train)
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
best_threshold = 0
best_f1 = 0
for threshold in np.arange(0, 1, 0.01):
    predicted_valid = probabilities_one_valid > threshold
    if f1_score(target_valid, predicted_valid) > best_f1:
        best_f1 = f1_score(target_valid, predicted_valid)
        best_threshold = threshold
display('best_f1', best_f1)
display('best_threshold', best_threshold)
display('AUC-ROC', roc_auc_score(target_valid, probabilities_one_valid))

'best_f1'

0.6684005201560469

'best_threshold'

0.51

'AUC-ROC'

0.8732278140292119

Вывод:

Обучил, используя в качестве борьбы с дисбалансом классов уравновешивание и метод upsampling по 3 модели: дерево решений, случайный лес и логистическую регрессию. 

Метод upsampling показал себя хуже, чем уравновешивание: результат оказался немного хуже.

Дерево решений и логистическая регрессия не дали приемлимого результата. Случайный лес оказался эффективнее.

С помощью изменения порога классификации удалось еще немного улучшить показатель F1 метрики у случайного леса.

## Тестирование модели

На тестовой выборке модель показала себя немного хуже, но все еще удовлетворяет требованиям. Получили высокие показатели F1 меры и AUC-ROC меры.

In [37]:
features_train2 = pd.concat([features_train, features_valid])
target_train2 = pd.concat([target_train, target_valid])
model = best_bforest_model
model.fit(features_train2, target_train2)
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
predicted_test = probabilities_one_test > 0.51
display('Показатель F1 меры на тестовой выборке:',f1_score(target_test, predicted_test))
display('Показатель AUC-ROC метрики:', roc_auc_score(target_test, probabilities_one_test))

'Показатель F1 меры на тестовой выборке:'

0.6203208556149733

'Показатель AUC-ROC метрики:'

0.8536842340397016

Конечный вывод:

    - Начальные данные практически без пропусков, что позволило быстрее приступить к решению задачи.
    
    - Обучил модель без учета дисбаланса классов для сравнения с другими моделями. Такая модель показала крайне низкий результат.
    
    - Обучил 6 моделей. По 3 модели с использованием метода уравновешнивания дисбаланса классов и метода upsampling: Дерево решений, логистическую регрессию и случайный лес.
    
    -Лучше всего себя показал случайный лес. Для большего улучшения результата поэкспериментировал с пороговыми значениями классификации, нашел оптимальное.
    
    -Стояла задача достичь показателя 0.59 F1 меры. В конечном счете удалось достичь высоких результатов F1 и AUC-ROC метрик: F1 = 0.62, AUC-ROC = 0.85.